In [ ]:

from google.colab import files
import networkx as nx
import io

uploaded = files.upload()

for filename in uploaded.keys():
    gml_bytes = uploaded[filename]
    gml_io = io.BytesIO(gml_bytes)
    G = nx.read_gml(gml_io)
    print(f"Loaded graph with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")


In [ ]:

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import ipywidgets as widgets
from IPython.display import display, clear_output

# Centralities
centralities = {
    'Degree': nx.degree_centrality(G),
    'Closeness': nx.closeness_centrality(G),
    'Betweenness': nx.betweenness_centrality(G),
    'Eigenvector': nx.eigenvector_centrality(G, max_iter=1000)
}

# Sliders for layout control
k_slider = widgets.FloatSlider(value=0.5, min=0.01, max=2.0, step=0.01, description='k:')
iter_slider = widgets.IntSlider(value=50, min=10, max=500, step=10, description='iterations:')
run_button = widgets.Button(description="Redraw Graphs")

# Output area
output = widgets.Output()

def draw_graph_with_layout(k_val, iter_val):
    pos = nx.spring_layout(G, seed=42, k=k_val, iterations=iter_val)

    fig, axs = plt.subplots(2, 2, figsize=(14, 12))
    axs = axs.flatten()

    for i, (name, values) in enumerate(centralities.items()):
        node_color = list(values.values())
        node_size = [300 + 2000 * v for v in node_color]
        norm = mcolors.Normalize(vmin=min(node_color), vmax=max(node_color))
        cmap = cm.viridis
        sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
        sm.set_array([])

        nx.draw(
            G, pos, ax=axs[i],
            with_labels=True,
            node_color=node_color,
            node_size=node_size,
            edge_color='gray',
            cmap=cmap
        )
        axs[i].set_title(f'{name} Centrality')
        plt.colorbar(sm, ax=axs[i])

    plt.tight_layout()
    plt.show()

def on_button_clicked(b):
    with output:
        clear_output(wait=True)
        draw_graph_with_layout(k_slider.value, iter_slider.value)

run_button.on_click(on_button_clicked)

# Display controls
display(widgets.HBox([k_slider, iter_slider, run_button]), output)

# Initial plot
with output:
    draw_graph_with_layout(k_slider.value, iter_slider.value)
